In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np


def generate_sequence(length, amplitude, frequency, phase, noise):
    x = np.linspace(0, 1, length)
    y_func = amplitude * np.sin(frequency * x - phase)
    y = y_func + noise * np.random.randn(x.shape[0])
    return y

def generate_data(dataset_size, seq_length, seed=0):
    np.random.seed(seed)
    seqs, labels = list(), list()
    label_numbers = np.arange(4).reshape([2, 2])
    for _ in range(dataset_size):
        amp_class = np.random.choice(2, 1, replace=False)
        if amp_class == 0:
            amplitude = np.random.normal(.75, .1)
        else:
            amplitude = np.random.normal(.1, .01)

        freq_class = np.random.choice(2, 1, replace=False)
        if freq_class == 0:
            frequency = np.random.normal(2 * np.pi, np.pi/4)
        else:
            frequency = np.random.normal(4 * np.pi, np.pi/4)
            
        phase = frequency * np.random.uniform(0, 1)    # cover a full period
        noise = amplitude * 0.15
        seq = generate_sequence(seq_length, amplitude, frequency, phase, noise)
        seqs.append(seq)
        labels.append(label_numbers[amp_class, freq_class])

    seqs = np.array(seqs)
    labels = np.array(labels)

    return seqs, labels

In [32]:
import plotly
from IPython.core.display import display, HTML
from IPython.display import Javascript
import json

plotly.offline.init_notebook_mode(connected=True)


def draw_js_plot(data):
    projection_plot, sequence_plot = data
    display(HTML("<div id=" + projection_plot['id'] + "></div>"))
    display(HTML("<div id=" + sequence_plot['id'] + "></div>"))

    js_code = '''
    require(['plotly'], function(Plotly) {

        var scatter_plot = function(element_id, x, y, customdata, colors, title) {
            var data = [{x:x, y:y, customdata:customdata, type:'scatter',
                     mode:'markers', marker:{size:5, color:colors}}],
                layout = {hovermode:'closest', title:title,
                    width: 900, height: 500};
            Plotly.newPlot(element_id, data, layout);
        };

        var proj_plot_id = ''' + projection_plot['id'] + ''',
            x = ''' + projection_plot['x'] + ''',
            y = ''' + projection_plot['y'] + ''',
            customdata = ''' + projection_plot['customdata'] + ''',
            colors = ''' + projection_plot['colors'] + ''',
            title = ''' + projection_plot['title'] + ''',
            proj_plot = document.getElementById(proj_plot_id);
        scatter_plot(proj_plot_id, x, y, customdata, colors, title);

        var seq_plot_id = ''' + sequence_plot['id'] + ''';
        scatter_plot(seq_plot_id, null, null, null, null, ''' + sequence_plot['title'] + ''');

        proj_plot.on('plotly_click', function(d) {
            customdata = d.points[0].customdata
            Plotly.animate(seq_plot_id, {
                data: [{y: customdata['sequence'],
                        marker:{size:5, color:customdata['color']}}],
                traces: [0],
                layout: {hovermode:'closest',
                    xaxis: {range: ''' + sequence_plot['x_range'] + '''},
                    yaxis: {range: ''' + sequence_plot['y_range'] + '''}
                }
            },
            {
                transition: {
                  duration: 200,
                  easing: 'cubic-in-out'
                }
            })
        });
    });
    '''
    return js_code


In [33]:
def format_plot_data(xy_coords, sequences, labels, title):
    x = xy_coords[:, 0].tolist()
    y = xy_coords[:, 1].tolist()
    sequences = sequences.tolist()

    labels = labels.flatten().tolist()
    label2color = ['red', 'blue', 'black', 'green']
    colors = [label2color[label] for label in labels]

    customdata = [{'sequence': seq, 'color': color} for seq, color in zip(sequences, colors)]

    projection_plot = {
        'id': json.dumps('proj_plot_' + title),
        'x': json.dumps(x),
        'y': json.dumps(y),
        'customdata': json.dumps(customdata),
        'colors': json.dumps(colors),
        'title': json.dumps('Projection: ' + title)
    }

    sequence_plot = {
        'id': json.dumps('seq_plot_' + title),
        'x_range': json.dumps([0, seq_length]),
        'y_range': json.dumps([-1.25, 1.25]),
        'title': json.dumps('Sequence: ' + title),
    }

    return projection_plot, sequence_plot

In [34]:
from sklearn.decomposition import TruncatedSVD

dataset_size = 500
seq_length = 50
sequences, labels = generate_data(dataset_size, seq_length)

pca_model = TruncatedSVD(n_components=2).fit(sequences)
xy_coords = pca_model.transform(sequences)

title = 'PCA'
plot_data = format_plot_data(xy_coords, sequences, labels, title)
Javascript(draw_js_plot(plot_data))

<IPython.core.display.Javascript object>

In [35]:
from sklearn.manifold import TSNE

tsne_model = TSNE(verbose=0, perplexity=30, min_grad_norm=1E-12, n_iter=2000, random_state=0)
xy_coords = tsne_model.fit_transform(sequences)

title = 'tSNE'
plot_data = format_plot_data(xy_coords, sequences, labels, title)
Javascript(draw_js_plot(plot_data))

<IPython.core.display.Javascript object>